In [1]:
import scipy.io
import numpy as np
import h5py
import mne
from mne import Epochs, create_info
from mne.io import RawArray
from mne.preprocessing import create_eog_epochs, ICA
from mne.channels import make_standard_montage
from mne.time_frequency import tfr_morlet
from mne.minimum_norm import apply_inverse_epochs, make_inverse_operator
from mne.datasets import sample
import matplotlib.pyplot as plt

Load the .mat file and extract EEG data

In [2]:
# Load the .mat file
mat = scipy.io.loadmat('EEG_Block01_20050215_084433.mat')

In [3]:
eeg_data = mat['raw_EEG']
sampling_rate = 1000

In [4]:
eeg_data = eeg_data[:-1]
len(eeg_data)

128

Create MNE Raw Object

In [5]:
ch_names = [f'EEG {i}' for i in range(1, eeg_data.shape[0] + 1)]
ch_types = ['eeg'] * len(ch_names)
info = create_info(ch_names=ch_names, sfreq=sampling_rate, ch_types=ch_types)
raw = RawArray(eeg_data, info)

Creating RawArray with float64 data, n_channels=128, n_times=396082
    Range : 0 ... 396081 =      0.000 ...   396.081 secs
Ready.


In [6]:
raw

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,128 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,500.00 Hz


Set the montage

In [7]:
montage = make_standard_montage('biosemi128')
montage_ch_names = montage.ch_names
rename_channels = {ch_names[i]: montage_ch_names[i] for i in range(len(ch_names))}
raw.rename_channels(rename_channels)
raw.set_montage(montage)

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,131 points
Good channels,128 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,500.00 Hz


Apply Bandpass and Notch Filters

In [8]:
raw.filter(1., 40., fir_design='firwin')  # Bandpass filter
raw.notch_filter(np.arange(50, 251, 50), fir_design='firwin')  # Notch filter to remove power line noise

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.5s


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,131 points
Good channels,128 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


Apply Hjorth-style Laplacian Spatial Filter

In [9]:
# Assuming that Hjorth-style Laplacian filter means surface Laplacian
# MNE does not have a direct implementation but can be done using surface Laplacian approach
from mne.preprocessing import compute_current_source_density
raw_csd = compute_current_source_density(raw)

Fitted sphere radius:         95.0 mm
Origin head coordinates:      0.0 0.0 40.1 mm
Origin device coordinates:    0.0 0.0 40.1 mm


Apply Common Average Reference

In [10]:
raw.set_eeg_reference('average', projection=True)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,131 points
Good channels,128 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


Cortical Source Estimation using sLORETA

In [14]:
import os.path as op

import mne
from mne.datasets import eegbci, fetch_fsaverage

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")

Using default location ~/mne_data for montage coregistration...
Creating ~/mne_data
179 files missing from root.txt in /Users/kana/mne_data/MNE-fsaverage-data


100%|████████████████████████████████████████| 196M/196M [00:00<00:00, 141GB/s]


Extracting missing files
Successfully extracted 179 files
10 files missing from bem.txt in /Users/kana/mne_data/MNE-fsaverage-data/fsaverage


100%|████████████████████████████████████████| 239M/239M [00:00<00:00, 209GB/s]


Extracting missing files
Successfully extracted 10 files


In [17]:
# Check that the locations of EEG electrodes is correct with respect to MRI
mne.viz.plot_alignment(
    raw.info,
    src=src,
    eeg=["original", "projected"],
    trans=trans,
    show_axes=True,
    mri_fiducials=True,
    dig="fiducials",
)

Reading /Users/kana/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
Using notebook 3d backend.

Using outer_skin.surf for head surface.
Channel types::	eeg: 128
Projecting sensors to the head surface


/opt/anaconda3/envs/py311/lib/python3.11/site-packages/pyvista/jupyter/notebook.py:37: UserWarning: Failed to use notebook backend: 

No module named 'trame'

Falling back to a static output.
  warnings.warn(


AttributeError: 'Image' object has no attribute 'value'